In [9]:
# import library
from sklearn import *
import pandas as pd

In [23]:
# load datasets
cancer = datasets.load_breast_cancer()
dtcancer = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
dttennis = pd.read_csv("../data/play_tennis.csv")

In [26]:
# split datasets
ctrain, ctest = model_selection.train_test_split(dtcancer, test_size=0.2, train_size=0.8, random_state=1)
tntrain, tntest = model_selection.train_test_split(dttennis, test_size=0.2, train_size=0.8, random_state=1)

In [27]:
# Decision Tree Classifier


In [28]:
# Id3 Estimator


In [29]:
# KMeans


In [30]:
# Logistic Regression


In [31]:
# Neural Network


In [ ]:
# SVM
